In [ ]:
from langgraph.graph import StateGraph, START, END
from langchain.chat_models import init_chat_model
from langgraph.graph.message import MessagesState
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_core.tools import tool
from langgraph.checkpoint.sqlite import SqliteSaver
import sqlite3
from langgraph.types import interrupt, Command


import os
from dotenv import load_dotenv

load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")

llm = init_chat_model("openai:gpt-4o-mini", openai_api_key=api_key)

conn = sqlite3.connect("memory.db", check_same_thread=False)

config = {
    "configurable": {
        "thread_id": "3",
    }
}

In [48]:
class State(MessagesState):
    custom_stuff: str


graph_builder = StateGraph(State)

In [41]:
@tool
def get_human_feedback(poem: str):
    """
    Asks the user for feedback on the poem.
    Use this before returning the final response.
    """
    feedback = interrupt(f"Here is the poem, tell me what you think\n{poem}")
    return feedback


llm_with_tools = llm.bind_tools(tools=[get_human_feedback])


def chatbot(state: State):
    response = llm_with_tools.invoke(
        f"""
        You are an expert in making poems.

        Use the `get_human_feedback` tool to get feedback on your poem.

        Only after you receive positive feedback you can return the final poem.

        ALWAYS ASK FOR FEEDBACK FIRST.

        Here is the conversation history:

        {state["messages"]}
    """
    )
    return {
        "messages": [response],
    }

In [49]:
tool_node = ToolNode(
    tools=[get_human_feedback],
)

graph_builder.add_node("chatbot", chatbot)
graph_builder.add_node("tools", tool_node)


graph_builder.add_edge(START, "chatbot")
graph_builder.add_conditional_edges("chatbot", tools_condition)
graph_builder.add_edge("tools", "chatbot")

graph = graph_builder.compile(
    checkpointer=SqliteSaver(conn),
)

In [50]:
result = graph.invoke(
    {
        "messages": [
            {"role": "user", "content": "Please make a poem about Python code."},
        ]
    },
    config=config,
)

In [51]:
for message in result["messages"]:
    message.pretty_print()

================================ Human Message =================================

Please make a poem about Python code
================================== Ai Message ==================================
Tool Calls:
  get_human_feedback (call_GzFf3VE4DFZA3YyVcqhRv6nw)
 Call ID: call_GzFf3VE4DFZA3YyVcqhRv6nw
  Args:
    poem: In the realm of code, where logic gleams,
Python weaves its magic, fulfilling dreams.
With every line, a purpose unfolds,
In elegant syntax, the story is told.

Indents like whispers, guiding the flow,
A dance of commands, where ideas can grow.
Loops spin like cycles, while functions embrace,
A tapestry woven, bringing grace to the space.

Modules aplenty, libraries wide,
In this digital world, Python’s our guide.
From data manipulation to web’s gentle might,
With Python beside us, we conquer the night.
================================ Human Message =================================

Write a poem about Python code
================================== Ai Message =========

In [53]:
snapshot = graph.get_state(config)

snapshot.next

('tools',)

In [56]:
feedback = Command(resume="It looks good!")

result = graph.invoke(
    feedback,
    config=config,
)

for message in result["messages"]:
    message.pretty_print()

================================ Human Message =================================

Please make a poem about Python code
================================== Ai Message ==================================
Tool Calls:
  get_human_feedback (call_GzFf3VE4DFZA3YyVcqhRv6nw)
 Call ID: call_GzFf3VE4DFZA3YyVcqhRv6nw
  Args:
    poem: In the realm of code, where logic gleams,
Python weaves its magic, fulfilling dreams.
With every line, a purpose unfolds,
In elegant syntax, the story is told.

Indents like whispers, guiding the flow,
A dance of commands, where ideas can grow.
Loops spin like cycles, while functions embrace,
A tapestry woven, bringing grace to the space.

Modules aplenty, libraries wide,
In this digital world, Python’s our guide.
From data manipulation to web’s gentle might,
With Python beside us, we conquer the night.
================================ Human Message =================================

Write a poem about Python code
================================== Ai Message =========